# Necessary Tools and Services

In [27]:
! pip install pandas
! pip install matplotlib
! pip install numpy
! pip install seaborn
! pip install unzip
! pip install gensim
! pip install nltk
! pip install wordcloud
! pip install spacy
! pip install spacy_download
! pip install pyLDAvis
! pip install PyStemmer

# Importing Necessary Libraries

In [2]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kha060/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/kha060/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kha060/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 56.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


# Data Preprocessing Functions

In [3]:
stopwords = stopwords.words('english')
stopwords.extend(['#', '`', '"', '@'])

#print (stopwords)

# Importing Dataset and extract necessary columns

In [4]:
df = pd.read_csv("Dataset/GitHubIssueList.csv")
df['texts'] = df['title']+' '+df['body']
column_names = ['texts']
df.to_csv("Dataset/GitHubDatasetforTopicModeling.csv", index=False, columns=column_names)


# new_df = pd.read_csv("Dataset/GitHubDatasetforTopicModeling.csv")

# new_df['texts'] =new_df['texts'].replace('\d+', '')
# new_df['texts']=new_df['texts'].apply(str)
# new_df['texts'] = new_df['texts'].map(lambda x: re.sub('[,\.!?]', '', x))
# ## Converting to Lower Case
# new_df['texts'] = new_df['texts'].map(lambda x: x.lower())

# data_text = new_df[['texts']]
# data_text['index'] = data_text.index
# documents = data_text

# print(len(documents))

In [5]:
new_df = pd.read_csv("Dataset/GitHubDatasetforTopicModeling.csv")
new_df['texts'] =new_df['texts'].replace('\d+', '')
new_df['texts']=new_df['texts'].apply(str)
new_df['texts'] = new_df['texts'].map(lambda x: re.sub('[,\.!?]', '', x))
## Converting to Lower Case
new_df['texts'] = new_df['texts'].map(lambda x: x.lower())


In [6]:
data = new_df.texts.values.tolist()
print (data[0][0:90])

schedule "@daily" is wrongly declared in the "dag/core concepts"  ### what do you see as a


In [7]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

schedule @daily wrongly declare core concept # see issue find small bug core concept docum


In [8]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['schedule', 'daily', 'wrongly', 'declare', 'core', 'concept', 'see', 'issue', 'find', 'small', 'bug', 'core', 'concept', 'documentation', 'regard', 'daily', 'schedule', 'apache', 'airflow', 'stable']


In [35]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 3), (5, 1), (6, 1), (7, 2), (8, 1), (9, 3), (10, 1), (11, 3), (12, 1), (13, 2), (14, 4), (15, 1), (16, 6), (17, 1), (18, 2), (19, 2)]
agree


In [36]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

# Vizualizing the Data

In [37]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=20)
vis

/home/kha060/LDATopicModelling/jup_notebook/lib/python3.10/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.480285 -0.145997       1        1  20.474150
16    -0.501969  0.054404       2        1  19.458377
9     -0.307764 -0.391023       3        1   9.091194
4     -0.108806 -0.473392       4        1   8.321085
11    -0.119941  0.493968       5        1   7.387257
14    -0.313898  0.358822       6        1   5.809813
19    -0.319702  0.179667       7        1   5.518892
17    -0.221859 -0.164022       8        1   4.482449
5      0.103998 -0.416942       9        1   4.137970
0      0.378773 -0.142558      10        1   3.739614
13     0.290800 -0.368031      11        1   3.673889
6      0.228500  0.414649      12        1   1.643766
2     -0.049307 -0.006038      13        1   1.486090
7      0.063408  0.013023      14        1   1.371990
8      0.059395  0.205529      15        1   0.814197
15     0.185051  0.211428      16        1   0.669265
10     0.192613 -0.069439      17        1   0.517083
12     0.241144  0.030213      18        1   0.509703
18     0.361623  0.047332      19        1   0.473805
1      0.318226  0.168407      20        1   0.419412, topic_info=           Term          Freq         Total Category  logprob  loglift
217        file  99978.000000  99978.000000  Default  20.0000  20.0000
234        info  64186.000000  64186.000000  Default  19.0000  19.0000
83        error  40437.000000  40437.000000  Default  18.0000  18.0000
111        test  22164.000000  22164.000000  Default  17.0000  17.0000
245        line  33399.000000  33399.000000  Default  16.0000  16.0000
...         ...           ...           ...      ...      ...      ...
15035  genotype      0.073393      2.723159  Topic20 -12.1136   1.8604
10408       vcf      0.073401      2.725222  Topic20 -12.1135   1.8597
17355        dp      0.073395      2.724651  Topic20 -12.1135   1.8598
6986   sequence      0.073390      2.722918  Topic20 -12.1136   1.8604
7670    variant      0.073390      2.724286  Topic20 -12.1136   1.8599

[531 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
419       1  0.095155    access
419       7  0.904440    access
169       7  0.999367   account
2264      6  0.999813        ad
73        5  0.999796       add
...     ...       ...       ...
168       4  0.020964      work
341       8  0.996462    worker
1820      1  0.999307  workflow
1235      1  0.999360     write
1181      2  0.999300     wrong

[551 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 17, 10, 5, 12, 15, 20, 18, 6, 1, 14, 7, 3, 8, 9, 16, 11, 13, 19, 2])

# Bigram and Trigram

In [38]:
# bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
# trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

# bigram = gensim.models.phrases.Phraser(bigram_phrases)
# trigram = gensim.models.phrases.Phraser(trigram_phrases)


# def make_bigrams(texts):
#     return (bigram[doc] for doc in texts)

# def make_trigrams(texts):
#     return (trigram[bigram[doc]] for doc in texts)

# data_bigrams = make_bigrams(data_words)
# data_bigrams_trigrams = make_trigrams(data_words)



In [39]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['schedule', 'daily', 'wrongly', 'declare', 'core', 'concept', 'see', 'issue', 'find', 'small', 'bug', 'core', 'concept', 'documentation', 'regard', 'daily', 'schedule', 'apache', 'airflow', 'stable', 'core', 'concept', 'dagshtml', 'running', 'dag', 'dag', 'require', 'schedule', 'very', 'common', 'define', 'one', 'define', 'schedule', 'argument', 'python', 'schedule', 'daily', 'schedule', 'argument', 'take', 'value', 'valid', 'cron', 'schedule', 'value', 'also', 'python', 'mistake', 'daily', 'notation', 'instead', 'otherwise', 'dag', 'run', 'hour', 'second', 'course', 'need', 'replace', 'hour', 'dag', 'run', 'daily', 'solve', 'problem', 'change', 'documentation', 'marked', 'location', 'schedule', 'argument', 'take', 'value', 'valid', 'cron', 'schedule', 'value', 'daily', 'run', 'also', 'python', 'else', 'response_willing_submit', 'willing_submit_pr', 'code_conduct', 'agree_follow_project', 'code', 'main']


# Remove Stopwords, Make Bigrams and Lemmatize

In [40]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
#     nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
#     texts_out = []
#     for text in texts:
#         doc = nlp(text)
#         new_text = []
#         for token in doc:
#             if token.pos_ in allowed_postags:
#                 new_text.append(token.lemma_)
#         final = " ".join(new_text)
#         texts_out.append(final)
#     return (texts_out)


# lemmatized_texts = lemmatization(data)
# print (lemmatized_texts[0][0:90])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [41]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['schedule', 'daily', 'wrongly', 'declare', 'core', 'concept', 'see', 'issue', 'find', 'small', 'bug', 'core', 'concept', 'documentation', 'regard', 'daily', 'schedule', 'stable', 'core', 'concept', 'dagshtml', 'run', 'require', 'schedule', 'common', 'define', 'define', 'schedule', 'argument', 'daily', 'schedule', 'argument', 'take', 'value', 'valid', 'cron', 'schedule', 'value', 'also', 'python', 'daily', 'notation', 'instead', 'otherwise', 'run', 'hour', 'second', 'course', 'need', 'replace', 'hour', 'run', 'daily', 'solve', 'problem', 'change', 'documentation', 'mark', 'location', 'schedule', 'argument', 'take', 'value', 'valid', 'cron', 'schedule', 'value', 'daily', 'run', 'also', 'python', 'else', 'response', 'willing_submit', 'willing_submit', 'pr', 'code_conduct', 'agree_follow', 'project', 'code', 'main']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [42]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 3), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 3), (9, 3), (10, 1), (11, 2), (12, 1), (13, 6), (14, 1), (15, 2), (16, 2), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 4), (37, 8), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 2), (45, 4), (46, 2), (47, 1)]]


In [43]:
id2word[0]

'agree_follow'

In [44]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('agree_follow', 1),
  ('also', 2),
  ('argument', 3),
  ('bug', 1),
  ('change', 1),
  ('code', 1),
  ('code_conduct', 1),
  ('common', 1),
  ('concept', 3),
  ('core', 3),
  ('course', 1),
  ('cron', 2),
  ('dagshtml', 1),
  ('daily', 6),
  ('declare', 1),
  ('define', 2),
  ('documentation', 2),
  ('else', 1),
  ('find', 1),
  ('hour', 2),
  ('instead', 1),
  ('issue', 1),
  ('location', 1),
  ('main', 1),
  ('mark', 1),
  ('need', 1),
  ('notation', 1),
  ('otherwise', 1),
  ('pr', 1),
  ('problem', 1),
  ('project', 1),
  ('python', 2),
  ('regard', 1),
  ('replace', 1),
  ('require', 1),
  ('response', 1),
  ('run', 4),
  ('schedule', 8),
  ('second', 1),
  ('see', 1),
  ('small', 1),
  ('solve', 1),
  ('stable', 1),
  ('take', 2),
  ('valid', 2),
  ('value', 4),
  ('willing_submit', 2),
  ('wrongly', 1)]]

In [45]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=17, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [46]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.230*"dataset" + 0.083*"observe" + 0.068*"identical" + 0.044*"collection" '
  '+ 0.033*"confuse" + 0.022*"history" + 0.020*"wrongly" + 0.000*"liger" + '
  '0.000*"gene" + 0.000*"variant"'),
 (1,
  '0.147*"level" + 0.125*"import" + 0.069*"range" + 0.053*"demo" + '
  '0.048*"reverse" + 0.046*"render" + 0.029*"minimal" + 0.023*"module" + '
  '0.019*"entirely" + 0.007*"topic"'),
 (2,
  '0.421*"name" + 0.108*"service" + 0.045*"label" + 0.025*"exec" + 0.023*"pid" '
  '+ 0.021*"bind" + 0.018*"namespace" + 0.013*"initialize" + 0.011*"volume" + '
  '0.010*"pool"'),
 (3,
  '0.215*"invenio" + 0.141*"version" + 0.130*"update" + 0.062*"release" + '
  '0.048*"late" + 0.030*"pull" + 0.023*"fix" + 0.020*"deprecate" + '
  '0.019*"https" + 0.017*"upgrade"'),
 (4,
  '0.199*"thread" + 0.087*"mismatch" + 0.079*"application" + 0.054*"provider" '
  '+ 0.034*"connect" + 0.027*"serialization" + 0.026*"connection" + '
  '0.014*"serialize" + 0.012*"simultaneously" + 0.011*"mysql"'),
 (5,
  '0.186*"direc

In [47]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.946505620008436

Coherence Score:  0.40924936263458234


# Visualize the topics-keywords

In [48]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.321972 -0.036615       1        1  36.311070
10    -0.290692 -0.103629       2        1  16.646734
7     -0.228731  0.237787       3        1   8.998712
8     -0.152254 -0.295777       4        1   7.111470
13    -0.224642  0.245483       5        1   6.499884
16    -0.042300 -0.019450       6        1   4.898189
2      0.106886  0.021079       7        1   4.109985
12     0.051624  0.005982       8        1   3.066901
3     -0.007957 -0.177842       9        1   2.450926
1      0.162469  0.028860      10        1   2.216204
5      0.145679  0.014503      11        1   2.048566
6      0.078850  0.009783      12        1   1.281170
11     0.102575  0.013077      13        1   1.145205
4      0.164067  0.014638      14        1   1.034211
9      0.169832  0.014599      15        1   0.909771
14     0.104266  0.013119      16        1   0.888644
0      0.182301  0.014402      17        1   0.382357, topic_info=           Term          Freq         Total Category  logprob  loglift
181        file  92423.000000  92423.000000  Default  30.0000  30.0000
193        info  50375.000000  50375.000000  Default  29.0000  29.0000
206        name  43063.000000  43063.000000  Default  28.0000  28.0000
201        line  29604.000000  29604.000000  Default  27.0000  27.0000
241        site  26859.000000  26859.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
5945    variant      0.083928      3.018672  Topic17 -11.6388   1.9840
6981   analysis      0.083481      2.975741  Topic17 -11.6441   1.9929
8104        vcf      0.083732      3.032054  Topic17 -11.6411   1.9772
13497        dp      0.083524      2.987064  Topic17 -11.6436   1.9897
8686      fastp      0.083489      2.996935  Topic17 -11.6440   1.9859

[684 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1375      6  0.997750     accord
722       6  0.998652     action
1655      2  0.999610         ad
62        1  0.999737        add
250       8  0.997618   addition
...     ...       ...        ...
881       4  0.999323   workflow
47       17  0.983885    wrongly
1394     13  0.775839        www
956      14  0.987097  yesterday
1182      3  0.993255      yield

[556 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 11, 8, 9, 14, 17, 3, 13, 4, 2, 6, 7, 12, 5, 10, 15, 1])

In [49]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=13, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [50]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.530374842674192

Coherence Score:  0.4040445083698645


In [51]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.281150 -0.060808       1        1  24.904265
8      0.283137  0.118280       2        1  20.183827
10     0.235917  0.036777       3        1  13.198703
7      0.129229 -0.167039       4        1   9.119026
2      0.088387  0.282330       5        1   6.796540
12    -0.018499 -0.192480       6        1   5.801803
9     -0.110038 -0.009123       7        1   5.416468
0     -0.031352 -0.210198       8        1   3.756021
1     -0.192083  0.052849       9        1   2.860982
6     -0.169351  0.048251      10        1   2.586640
4     -0.096034 -0.009640      11        1   1.923840
5     -0.208755  0.055969      12        1   1.735341
3     -0.191708  0.054830      13        1   1.716543, topic_info=             Term          Freq         Total Category  logprob  loglift
181          file  82677.000000  82677.000000  Default  30.0000  30.0000
99           type  30897.000000  30897.000000  Default  29.0000  29.0000
193          info  31495.000000  31495.000000  Default  28.0000  28.0000
915        sample  16990.000000  16990.000000  Default  27.0000  27.0000
7698      bcftool  48201.000000  48201.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
1068      rewrite    233.986401    236.250646  Topic13  -5.2074   4.0552
1800      keyword    214.824440    217.088691  Topic13  -5.2929   4.0544
1491  deprecation    210.131125    212.395369  Topic13  -5.3150   4.0541
190          http   1256.481893   1306.248846  Topic13  -3.5266   4.0260
1215       engine    457.540740    644.365296  Topic13  -4.5368   3.7225

[492 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
477       5  0.997981         _
6865      9  0.997643     abort
2239     10  0.998726    accept
1655      2  0.999727        ad
62        1  0.999727       add
...     ...       ...       ...
881       2  0.999243  workflow
533      11  0.983741   wrapper
9662     12  0.983107  writable
995       5  0.999392     write
47       11  0.987195   wrongly

[550 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 9, 11, 8, 3, 13, 10, 1, 2, 7, 5, 6, 4])

In [52]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=11, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -8.882852540565116

Coherence Score:  0.43417681983347367


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.277129  0.129683       1        1  28.659941
10    -0.247237 -0.183316       2        1  18.235222
0     -0.227937  0.269898       3        1  16.774553
7     -0.110358 -0.338170       4        1   8.189489
9      0.137512  0.014613       5        1   7.185368
2     -0.036427  0.004028       6        1   5.763092
4      0.041291  0.039104       7        1   4.443403
5      0.164499  0.003668       8        1   3.788550
3      0.170985  0.030557       9        1   3.440474
6      0.185423  0.032786      10        1   1.843634
1      0.199378 -0.002852      11        1   1.676275, topic_info=          Term          Freq         Total Category  logprob  loglift
193       info  42151.000000  42151.000000  Default  30.0000  30.0000
7698   bcftool  49390.000000  49390.000000  Default  29.0000  29.0000
181       file  63695.000000  63695.000000  Default  28.0000  28.0000
206       name  26531.000000  26531.000000  Default  27.0000  27.0000
201       line  18861.000000  18861.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
1069   snippet    124.090499    125.932238  Topic11  -5.8180   4.0739
14039       nf    122.398176    124.240346  Topic11  -5.8317   4.0737
5203      eval    120.069870    121.912549  Topic11  -5.8509   4.0734
874     module   4438.364460   5250.777967  Topic11  -2.2409   3.9205
2275    invoke    153.830927    387.343645  Topic11  -5.6031   3.1651

[464 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
13764      8  0.994864  aborted
2239       7  0.995584   accept
336        1  0.645860   access
336        5  0.271359   access
336        9  0.082308   access
...      ...       ...      ...
273        8  0.995193   worker
995        6  0.999368    write
955        1  0.999296    wrong
1182       4  0.991833    yield
8865       8  0.995459       œa

[566 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 11, 1, 8, 10, 3, 5, 6, 4, 7, 2])